In [1]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r "./drive/My Drive/task4" "."

!cd task4

Mounted at /content/drive


In [2]:
pip -q install rdkit-pypi

     |████████████████████████████████| 22.7 MB 1.7 MB/s 


In [4]:
import numpy as np
import pandas as pd
from rdkit.Chem import AllChem, MolFromSmiles
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow import keras

df = pd.read_csv("task4/pretrain_features.csv")
dl = pd.read_csv("task4/pretrain_labels.csv")
trf = pd.read_csv("task4/train_features.csv")
trl = pd.read_csv("task4/train_labels.csv")
tsf = pd.read_csv("task4/test_features.csv")

ids = tsf['Id']

features = df.drop(columns = ["Id", "smiles"])
features2 = trf.drop(columns = ["Id", "smiles"])
features3 = tsf.drop(columns = ["Id", "smiles"])

In [5]:
#pretrained features
smiles_list = df['smiles'].to_list()
rdkit_mols = [MolFromSmiles(smiles) for smiles in smiles_list]
X = pd.DataFrame([np.array(AllChem.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=2048)) for mol in rdkit_mols], columns = ["smiles" + str(i) for i in range(2048)])
y = np.array(dl['lumo_energy'])
finalX = pd.concat([X, features ], axis = 1)

#fe
smiles_list2 = trf['smiles'].to_list()
rdkit_mols2 = [MolFromSmiles(smiles) for smiles in smiles_list2]
X2 = pd.DataFrame([np.array(AllChem.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=2048)) for mol in rdkit_mols2], columns = ["smiles" + str(i) for i in range(2048)])
y2 = np.array(trl['homo_lumo_gap'])
finalX2 = pd.concat([X2, features2], axis = 1)

smiles_list3 = tsf['smiles'].tolist()
rdkit_mols3 = [MolFromSmiles(smiles) for smiles in smiles_list3]
X3 = pd.DataFrame([np.array(AllChem.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=2048)) for mol in rdkit_mols3], columns = ["smiles" + str(i) for i in range(2048)])
finalX3 = pd.concat([X3, features3], axis = 1)

In [50]:
pretrain_mod = keras.Sequential([
                                 keras.layers.Dense(3000, activation = "relu" , input_dim = 3048),
                                 keras.layers.Dense(2000, activation = "relu"),
                                 keras.layers.Dense(1000, activation = "sigmoid"),
                                 keras.layers.Dense(500, activation = "linear"),
                                 keras.layers.Dense(1, activation = "linear")
])

In [51]:
pretrain_mod.compile(optimizer = tf.optimizers.Adam(0.001), loss = "mse", metrics = [keras.metrics.RootMeanSquaredError()])

In [ ]:
pretrain_mod.fit(finalX, y, validation_split = 0.2, shuffle = True, epochs = 15)

In [53]:
pretrain_mod.layers.pop()
pretrain_mod.layers.pop()
pretrain_mod.add(keras.layers.Dense(500, activation = "linear"))
pretrain_mod.add(keras.layers.Dense(1, activation = "linear"))

pretrain_mod.trainable = True


pretrain_mod.compile(optimizer = tf.optimizers.Adam(0.001), loss = "mse", metrics = [keras.metrics.RootMeanSquaredError()])

In [ ]:
pretrain_mod.fit(finalX2, y2, validation_split = 0.05, shuffle = True, epochs = 500 )

In [55]:

for layer in pretrain_mod.layers[:-2]:
  layer.trainable = False

pretrain_mod.compile(optimizer = tf.optimizers.Adam(0.00001), loss = "mse", metrics = [keras.metrics.RootMeanSquaredError()])

In [ ]:
pretrain_mod.fit(finalX2, y2, validation_split = 0.05, shuffle = True, epochs = 1500 )

In [48]:
fin = pretrain_mod.predict(finalX3)

In [49]:
finpd = pd.DataFrame(fin, columns = ["y"])
kms = pd.concat([ids, finpd], axis = 1)
kms.to_csv("final_output.csv", index = False)